In [ ]:
from google.colab import drive
import os

In [ ]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!cp --verbose gdrive/MyDrive/yelp_task/photo_tars/br_photos.tar br_photos.tar

'gdrive/MyDrive/yelp_task/photo_tars/br_photos.tar' -> 'br_photos.tar'


In [ ]:
!tar -xf br_photos.tar

In [ ]:
!rm br_photos.tar

In [ ]:
assert len(os.listdir('photos')) == 59556

In [ ]:
!pip install pytorch-lightning

     |████████████████████████████████| 849kB 4.0MB/s 
     |████████████████████████████████| 184kB 8.7MB/s 
     |████████████████████████████████| 829kB 8.2MB/s 
     |████████████████████████████████| 276kB 18.8MB/s 
     |████████████████████████████████| 112kB 19.1MB/s 
     |████████████████████████████████| 1.3MB 15.2MB/s 
     |████████████████████████████████| 143kB 31.3MB/s 
     |████████████████████████████████| 296kB 19.7MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=78d37fa0b6939fc7a11fbd49d05dd1396f9a28e69db545d408cfff7b012205fc
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=9ca4370af90ed30629f927202c3d956c737ffbfe3ae272c23b6bfbc66a021707
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built future P

In [ ]:
!nvidia-smi -L

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
from __future__ import print_function
from __future__ import division

In [ ]:
import torch
from torch.utils.data import DataLoader

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

In [ ]:
import torchvision
from torchvision import transforms

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import time
#import os
import copy
from tqdm.notebook import tqdm

In [ ]:
from gdrive.MyDrive.yelp_task.lightning_model import LightningTransfer, init_cls_model
from gdrive.MyDrive.yelp_task.dataset_loaders import IsRestaurantDataset

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
print("PyTorch Lightning Version: ",pl.__version__)

PyTorch Version:  1.8.1+cu101
Torchvision Version:  0.9.1+cu101
PyTorch Lightning Version:  1.2.10


In [ ]:
photo_dir = 'photos/'
csv_path = "gdrive/MyDrive/yelp_task/yelp_data/business_restaurant.csv"

In [ ]:
FLAGS = {
    'model_name': 'resnet',
    'num_classes': 2,
    'batch_size': 512,  # 512 / N gpu
    'num_workers': 4,  # 4 per gpu
    'learning_rate': 0.02,  # 0.02 * N gpu
    'max_epochs': 10,  # arbitrary
    'feature_extract': True,
    'use_pretrained': True
        }

In [ ]:
transfer_model, input_size = init_cls_model(FLAGS)

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'dev': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
print("Initializing Datasets and Dataloaders...")

Initializing Datasets and Dataloaders...


In [ ]:
csv_file = pd.read_csv(csv_path)

In [ ]:
csv_file.head()

,photo_id,is_business
0,3V7tgMx3Qw5L9ZjRLNbthA,True
1,fZo1owoYqwAHW7uZlTz1XQ,False
2,zwOCQ8w3gFuF3zi_dyIWpw,False
3,hQBfeDngFMpB9HX2CPKtag,True
4,Fjh4N5B38vJWVbuQk-v3aQ,True


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(csv_file, test_size=0.2, random_state=42, stratify=csv_file.is_business)

In [ ]:
dev, test = train_test_split(test, test_size=0.5, random_state=42, stratify=test.is_business)

In [ ]:
data_frames = {'train': train,
              'dev': dev,
              'test': test}

In [ ]:
# Create training and validation datasets
image_datasets = {x: IsRestaurantDataset(data_frames[x], photo_dir, data_transforms[x]) for x in ['train', 'dev', 'test']}
# Create training and validation dataloaders
dataloaders_dict = {x: DataLoader(image_datasets[x],
                                  batch_size=FLAGS['batch_size'],
                                  shuffle=True,
                                  num_workers=FLAGS['num_workers'],
                                  pin_memory=True) for x in ['train', 'dev']}
dataloaders_dict['test'] = DataLoader(image_datasets['test'],
                                      batch_size=FLAGS['batch_size'],
                                      shuffle=False,
                                      num_workers=FLAGS['num_workers'],
                                      pin_memory=True)

In [ ]:
dataloaders_dict

{'dev': <torch.utils.data.dataloader.DataLoader at 0x7f7fe1130ad0>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7f7fe1130a90>,
 'train': <torch.utils.data.dataloader.DataLoader at 0x7f7fe1130890>}

In [ ]:
checkpoint_callback = ModelCheckpoint(monitor='val_loss')
early_stop_callback = EarlyStopping(monitor='val_loss', patience=2)

In [ ]:
trainer = pl.Trainer(callbacks=[checkpoint_callback, early_stop_callback],
                     progress_bar_refresh_rate=1,)
                  #   stochastic_weight_avg=True, precision=16, max_epochs=FLAGS['max_epochs'], gpus=-1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [ ]:
trainer.fit(transfer_model, dataloaders_dict['train'], dataloaders_dict['dev'])


  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11.2 M
---------------------------------
1.0 K     Trainable params
11.2 M    Non-trainable params
11.2 M    Total params
44.710    Total estimated model params size (MB)


1

In [ ]:
trainer.save_checkpoint(f"gdrive/MyDrive/yelp_task/yelp_data/checkpoints/{FLAGS['model_name']}.ckpt")

In [ ]:
print(checkpoint_callback.best_model_path)

/content/lightning_logs/version_0/checkpoints/epoch=2-step=281.ckpt


In [ ]:
transfer_model, _ = init_cls_model(FLAGS, checkpoint_callback.best_model_path)

In [ ]:
trainer.test(transfer_model, dataloaders_dict['test'])

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.7108507752418518}
--------------------------------------------------------------------------------


[{'test_loss': 0.7108507752418518}]

In [ ]:
model = transfer_model.eval().cuda(device=0)

In [ ]:
preds, labels = [], []

In [ ]:
for image, is_restaurant in tqdm(image_datasets['test']):
    output = model(image.unsqueeze(0).cuda(0)).data.cpu().numpy()
    preds.append(output[0])
    labels.append(is_restaurant)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
accuracy_score(labels, preds)

0.6633646742780389

In [ ]:
f1_score(labels, preds)

0.6895804304071836